In [29]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [31]:
tf.keras.backend.clear_session()

In [32]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [33]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.598394,0.766362,0.669899,39.798880,0.638715,1.317344e+08,8.384593e+06,8.566149,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,151.297
1,2003-2,0.301903,0.598795,0.766745,0.670210,39.480034,0.638948,1.318964e+08,8.391946e+06,8.569210,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,138.707
2,2003-3,0.303709,0.599196,0.767127,0.670521,39.400256,0.639181,1.320584e+08,8.399299e+06,8.572270,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,135.009
3,2003-4,0.305311,0.599597,0.767509,0.670831,39.417185,0.639414,1.322204e+08,8.406652e+06,8.575331,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,126.554
4,2003-5,0.306860,0.599998,0.767892,0.671142,39.479943,0.639647,1.323824e+08,8.414005e+06,8.578392,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,0.596178,NaN,NaN,NaN,105.752943,0.659315,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,105.537361,0.659395,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,104.971241,0.659476,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310.845
239,2022-12,0.589305,NaN,NaN,NaN,104.857141,0.659557,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310.845


In [34]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.436732,-1.890989,-2.630882,-2.195846,-1.793459,-2.210000,-1.766933,-0.695996,-2.315693,-2.238131,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.404185,-1.868832,-2.578239,-2.161313,-1.806616,-2.177528,-1.747868,-0.654222,-2.273375,-2.196868,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.375442,-1.846676,-2.525597,-2.126781,-1.809908,-2.145055,-1.728803,-0.612447,-2.231056,-2.155605,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.349941,-1.824519,-2.472955,-2.092248,-1.809210,-2.112583,-1.709739,-0.570673,-2.188738,-2.114342,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.325290,-1.802362,-2.420313,-2.057715,-1.806620,-2.080111,-1.690674,-0.528898,-2.146420,-2.073079,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,1.208211,0.651753,1.041952,0.619029,0.966650,0.865987,-1.416165,0.555618,0.463875,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.873607,1.201241,0.624227,1.026783,0.635352,0.964148,0.848481,-1.404857,0.531641,0.443334,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.901718,1.194270,0.596702,1.011615,0.652040,0.961646,0.830975,-1.393550,0.507664,0.422793,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.928136,1.187300,0.569176,0.996447,0.638794,0.959144,0.813469,-1.382243,0.483687,0.402252,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [35]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
236    347.901
237    310.845
238    310.845
239        NaN
240        NaN
Name: Bahia - Consumo de Cimento (t), Length: 241, dtype: float64

In [36]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.436732,-1.890989,-2.630882,-2.195846,-1.793459,-2.210000,-1.766933,-0.695996,-2.315693,-2.238131,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.404185,-1.868832,-2.578239,-2.161313,-1.806616,-2.177528,-1.747868,-0.654222,-2.273375,-2.196868,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.375442,-1.846676,-2.525597,-2.126781,-1.809908,-2.145055,-1.728803,-0.612447,-2.231056,-2.155605,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.349941,-1.824519,-2.472955,-2.092248,-1.809210,-2.112583,-1.709739,-0.570673,-2.188738,-2.114342,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.325290,-1.802362,-2.420313,-2.057715,-1.806620,-2.080111,-1.690674,-0.528898,-2.146420,-2.073079,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.005306,1.275616,1.370224,1.339087,0.469766,1.171052,1.163205,-1.340370,1.005749,1.008566,...,1.272681,1.521503,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.029532,1.275493,1.358762,1.336451,0.457435,1.163001,1.164562,-1.364998,1.008863,1.002613,...,1.288866,1.512161,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.057198,1.275369,1.347300,1.333816,0.454716,1.154950,1.165920,-1.389626,1.011977,0.996661,...,1.304998,1.500026,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.082730,1.275246,1.335837,1.331180,0.446427,1.146900,1.167277,-1.414254,1.015092,0.990708,...,1.332157,1.486995,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [37]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
163    280.074
164    277.938
165    275.802
166    273.666
167    271.530
Name: Bahia - Consumo de Cimento (t), Length: 168, dtype: float64

In [38]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [39]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, start_index)
reshaped_train.shape

(132, 36, 64)

In [40]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
132,0.543796,0.557829,0.275687,0.450270,1.069363,0.629694,0.900016,0.005667,0.917759,1.088312,...,0.467927,0.897908,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401
133,0.564912,0.566926,0.264670,0.450043,1.054155,0.622035,0.914134,-0.050121,0.925696,1.092431,...,0.505071,0.970274,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620
134,0.584512,0.576023,0.253652,0.449817,1.039972,0.614376,0.928253,-0.105908,0.933634,1.096551,...,0.541297,1.040830,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284
135,0.603507,0.585120,0.242635,0.449591,1.025069,0.606718,0.942371,-0.161695,0.941571,1.100670,...,0.571731,1.107516,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516
136,0.624844,0.594217,0.231618,0.449365,1.009687,0.599059,0.956489,-0.217482,0.949509,1.104789,...,0.611614,1.168977,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,1.208211,0.651753,1.041952,0.619029,0.966650,0.865987,-1.416165,0.555618,0.463875,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.873607,1.201241,0.624227,1.026783,0.635352,0.964148,0.848481,-1.404857,0.531641,0.443334,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.901718,1.194270,0.596702,1.011615,0.652040,0.961646,0.830975,-1.393550,0.507664,0.422793,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.928136,1.187300,0.569176,0.996447,0.638794,0.959144,0.813469,-1.382243,0.483687,0.402252,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [41]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 64)

In [42]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [43]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [44]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [45]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3100517321, 2619655037, 2475093570, 2858724758, 500304646, 2681065605, 143858914, 2947457618, 3657861112, 2129357278, 2882651845, 3657967743, 478942595, 1696033779, 3131986710, 681334674, 1982898026, 3091299421, 1415368275, 3848019143, 3022211047, 2678760028, 2508996446, 2752978405, 2404295195]


Step: 0 ___________________________________________
val_loss: 94.02339172363281
winner_seed: 3100517321


Step: 1 ___________________________________________
val_loss: 77.81446838378906
winner_seed: 2619655037


Step: 2 ___________________________________________
val_loss: 83.1001205444336


Step: 3 ___________________________________________
val_loss: 90.46727752685547


Step: 4 ___________________________________________
val_loss: 76.70256805419922
winner_seed: 500304646


Step: 5 ___________________________________________
val_loss: 82.28694152832031


Step: 6 ___________________________________________
val_loss: 1337.5093994140625


Step: 7 ___________________________________________
val_

In [46]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 111ms/step - loss: 52138.6094 - val_loss: 3013.5142
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 4055.8674 - val_loss: 3742.0610
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 3442.5247 - val_loss: 2974.3169
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 3555.3809 - val_loss: 4151.2822
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 4312.2251 - val_loss: 5961.6987
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 3922.6702 - val_loss: 3677.2725
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 4790.4521 - val_loss: 5641.3682
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 4597.8315 - val_loss: 3888.2302
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 3749.8992 - val_loss: 2534.3298
Epoch 10/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 379.8127 - val_loss: 232.3594
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 398.5129 - val_loss: 350.8828
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 504.5273 - val_loss: 540.1118
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 437.6326 - val_loss: 275.7986
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 482.5293 - val_loss: 448.0541
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 529.7239 - val_loss: 319.4756
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 572.9568 - val_loss: 282.8953
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 470.8848 - val_loss: 222.5961
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 419.0419 - val_loss: 313.2856
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 388.0

4/4 [==============================] - 0s 8ms/step - loss: 342.3730 - val_loss: 136.8564
Epoch 158/10000
4/4 [==============================] - 0s 7ms/step - loss: 311.1884 - val_loss: 198.6462
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 303.5992 - val_loss: 195.0752
Epoch 160/10000
4/4 [==============================] - 0s 8ms/step - loss: 289.8568 - val_loss: 351.9517
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 419.4721 - val_loss: 332.1593
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 349.0553 - val_loss: 275.3267
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 344.9890 - val_loss: 167.0590
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 324.0639 - val_loss: 368.8037
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 352.4156 - val_loss: 219.4496
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 320.3865 - val_loss: 136.4651
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 305.3598 - val_loss: 194.1058
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 337.6782 - val_loss: 218.0255
Epoch 238/10000
4/4 [==============================] - 0s 8ms/step - loss: 323.8937 - val_loss: 285.8318
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 395.2963 - val_loss: 175.9548
Epoch 240/10000
4/4 [==============================] - 0s 8ms/step - loss: 374.7729 - val_loss: 224.2930
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 511.2527 - val_loss: 138.1465
Epoch 242/10000
4/4 [==============================] - 0s 8ms/step - loss: 345.0288 - val_loss: 162.9046
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 396.7766 - val_loss: 133.8516
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 301.3719 - val_loss: 169.8255
Epoch 314/10000
4/4 [==============================] - 0s 7ms/step - loss: 294.1856 - val_loss: 224.8671
Epoch 315/10000
4/4 [==============================] - 0s 8ms/step - loss: 311.7937 - val_loss: 171.7636
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 362.1684 - val_loss: 152.2631
Epoch 317/10000
4/4 [==============================] - 0s 7ms/step - loss: 337.1411 - val_loss: 245.5936
Epoch 318/10000
4/4 [==============================] - 0s 8ms/step - loss: 304.9753 - val_loss: 196.5519
Epoch 319/10000
4/4 [==============================] - 0s 8ms/step - loss: 348.1430 - val_loss: 283.2158
Epoch 320/10000
4/4 [==============================] - 0s 7ms/step - loss: 318.3077 - val_loss: 165.6278
Epoch 321/10000
4/4 [==============================] - 0s 8ms/step - loss: 325.7601 - val_loss: 148.6074
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - lo

...

4/4 [==============================] - 0s 8ms/step - loss: 221.2256 - val_loss: 176.7335
Epoch 1170/10000
4/4 [==============================] - 0s 7ms/step - loss: 234.9480 - val_loss: 135.3707
Epoch 1171/10000
4/4 [==============================] - 0s 8ms/step - loss: 228.7622 - val_loss: 225.2308
Epoch 1172/10000
4/4 [==============================] - 0s 8ms/step - loss: 240.1259 - val_loss: 175.2139
Epoch 1173/10000
4/4 [==============================] - 0s 7ms/step - loss: 238.5454 - val_loss: 289.1403
Epoch 1174/10000
4/4 [==============================] - 0s 8ms/step - loss: 215.5915 - val_loss: 213.8705
Epoch 1175/10000
4/4 [==============================] - 0s 8ms/step - loss: 221.1900 - val_loss: 195.1371
Epoch 1176/10000
4/4 [==============================] - 0s 7ms/step - loss: 217.4462 - val_loss: 212.7382
Epoch 1177/10000
4/4 [==============================] - 0s 8ms/step - loss: 225.8491 - val_loss: 177.4368
Epoch 1178/10000
4/4 [==============================] - 0s 7ms/

4/4 [==============================] - 0s 7ms/step - loss: 253.6358 - val_loss: 151.3555
Epoch 1247/10000
4/4 [==============================] - 0s 8ms/step - loss: 285.5504 - val_loss: 134.5775
Epoch 1248/10000
4/4 [==============================] - 0s 8ms/step - loss: 227.2930 - val_loss: 175.6319
Epoch 1249/10000
4/4 [==============================] - 0s 8ms/step - loss: 242.1335 - val_loss: 160.0287
Epoch 1250/10000
4/4 [==============================] - 0s 8ms/step - loss: 227.4490 - val_loss: 195.9426
Epoch 1251/10000
4/4 [==============================] - 0s 8ms/step - loss: 268.6923 - val_loss: 383.8959
Epoch 1252/10000
4/4 [==============================] - 0s 7ms/step - loss: 305.3732 - val_loss: 210.9463
Epoch 1253/10000
4/4 [==============================] - 0s 8ms/step - loss: 250.3868 - val_loss: 173.0524
Epoch 1254/10000
4/4 [==============================] - 0s 8ms/step - loss: 222.5603 - val_loss: 213.2210
Epoch 1255/10000
4/4 [==============================] - 0s 7ms/

4/4 [==============================] - 0s 8ms/step - loss: 219.2343 - val_loss: 160.0180
Epoch 1324/10000
4/4 [==============================] - 0s 8ms/step - loss: 186.9699 - val_loss: 154.4865
Epoch 1325/10000
4/4 [==============================] - 0s 7ms/step - loss: 213.1927 - val_loss: 163.9074
Epoch 1326/10000
4/4 [==============================] - 0s 8ms/step - loss: 247.0844 - val_loss: 194.7978
Epoch 1327/10000
4/4 [==============================] - 0s 8ms/step - loss: 241.8287 - val_loss: 225.2939
Epoch 1328/10000
4/4 [==============================] - 0s 7ms/step - loss: 233.0544 - val_loss: 331.9968
Epoch 1329/10000
4/4 [==============================] - 0s 8ms/step - loss: 256.4839 - val_loss: 223.8694
Epoch 1330/10000
4/4 [==============================] - 0s 8ms/step - loss: 215.8992 - val_loss: 161.1281
Epoch 1331/10000
4/4 [==============================] - 0s 7ms/step - loss: 221.3852 - val_loss: 332.2769
Epoch 1332/10000
4/4 [==============================] - 0s 8ms/

4/4 [==============================] - 0s 7ms/step - loss: 233.7778 - val_loss: 163.5399
Epoch 1401/10000
4/4 [==============================] - 0s 8ms/step - loss: 221.6766 - val_loss: 155.6652
Epoch 1402/10000
4/4 [==============================] - 0s 8ms/step - loss: 242.9051 - val_loss: 221.6344
Epoch 1403/10000
4/4 [==============================] - 0s 8ms/step - loss: 227.1725 - val_loss: 209.5746
Epoch 1404/10000
4/4 [==============================] - 0s 8ms/step - loss: 223.0731 - val_loss: 274.3691
Epoch 1405/10000
4/4 [==============================] - 0s 7ms/step - loss: 241.2582 - val_loss: 401.6643
Epoch 1406/10000
4/4 [==============================] - 0s 8ms/step - loss: 217.0640 - val_loss: 174.2569
Epoch 1407/10000
4/4 [==============================] - 0s 8ms/step - loss: 258.8607 - val_loss: 156.1439
Epoch 1408/10000
4/4 [==============================] - 0s 7ms/step - loss: 277.2798 - val_loss: 191.4825
Epoch 1409/10000
4/4 [==============================] - 0s 8ms/

4/4 [==============================] - 0s 7ms/step - loss: 201.6561 - val_loss: 176.4471
Epoch 1478/10000
4/4 [==============================] - 0s 8ms/step - loss: 233.4221 - val_loss: 178.7753
Epoch 1479/10000
4/4 [==============================] - 0s 8ms/step - loss: 230.8534 - val_loss: 167.1460
Epoch 1480/10000
4/4 [==============================] - 0s 7ms/step - loss: 220.6346 - val_loss: 148.0049
Epoch 1481/10000
4/4 [==============================] - 0s 8ms/step - loss: 210.8822 - val_loss: 197.6100
Epoch 1482/10000
4/4 [==============================] - 0s 8ms/step - loss: 197.4177 - val_loss: 169.0541
Epoch 1483/10000
4/4 [==============================] - 0s 7ms/step - loss: 211.4410 - val_loss: 180.8451
Epoch 1484/10000
4/4 [==============================] - 0s 8ms/step - loss: 203.1860 - val_loss: 151.5624
Epoch 1485/10000
4/4 [==============================] - 0s 7ms/step - loss: 195.9032 - val_loss: 147.9672
Epoch 1486/10000
4/4 [==============================] - 0s 7ms/

In [47]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [48]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 280ms/step
Month-1 - Error: [[64.801926]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[7.0793457]]
1/1 [==============================] - 0s 14ms/step
Month-3 - Error: [[59.951553]]
1/1 [==============================] - 0s 14ms/step
Month-4 - Error: [[13.03772]]
1/1 [==============================] - 0s 14ms/step
Month-5 - Error: [[40.20735]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[31.629623]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[4.3067017]]
1/1 [==============================] - 0s 14ms/step
Month-8 - Error: [[10.32254]]
1/1 [==============================] - 0s 14ms/step
Month-9 - Error: [[2.0861206]]
1/1 [==============================] - 0s 14ms/step
Month-10 - Error: [[22.42694]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[47.22856]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error: [[9.976807]]
1/1 [=

In [49]:
display(mae)
display(mape)

27.163008

0.112763524

In [50]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [51]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[3359.0027]] - Target[3060.1059999999998]| =  Error: [[298.89673]]; MAPE:[[0.09767529]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[3223.9392]] - Target[3013.2439999999997]| =  Error: [[210.69531]]; MAPE:[[0.06992309]]
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[3159.4568]] - Target[3254.4680000000003]| =  Error: [[95.01123]]; MAPE:[[0.02919409]]


[array([[298.89673]], dtype=float32),
 array([[210.69531]], dtype=float32),
 array([[95.01123]], dtype=float32)]

201.53442

0.06559748